## 分析国家GDP 与 人口数 与最近五年获得的奥运奖牌数总和之间的函数关系

##　加载数据

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [9]:
%cd dataset

/content/drive/MyDrive/Colab Notebooks/dataset


In [10]:
!ls

data_dictionary.csv	summerOly_hosts.csv	    summerOly_programs.csv
summerOly_athletes.csv	summerOly_medal_counts.csv


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
athletes = pd.read_csv('summerOly_athletes.csv')
hosts = pd.read_csv('summerOly_hosts.csv')
medal_counts = pd.read_csv('summerOly_medal_counts.csv')
programs = pd.read_csv('summerOly_programs.csv',encoding='latin-1')
countrys = pd.read_csv('country.csv')

In [15]:
countrys

,Country,Code,Population,GDP per Capita
0,Afghanistan,AFG,32526562.0,594.323081
1,Albania,ALB,2889167.0,3945.217582
2,Algeria,ALG,39666519.0,4206.031232
3,American Samoa*,ASA,55538.0,NaN
4,Andorra,AND,70473.0,NaN
...,...,...,...,...
196,Vietnam,VIE,91703800.0,2111.138024
197,Virgin Islands*,ISV,103574.0,NaN
198,Yemen,YEM,26832215.0,1406.291651
199,Zambia,ZAM,16211767.0,1304.879014


In [18]:
countrys.info()
medal_counts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         201 non-null    object 
 1   Code            201 non-null    object 
 2   Population      196 non-null    float64
 3   GDP per Capita  176 non-null    float64
dtypes: float64(2), object(2)
memory usage: 6.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1435 entries, 0 to 1434
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rank    1435 non-null   int64 
 1   NOC     1435 non-null   object
 2   Gold    1435 non-null   int64 
 3   Silver  1435 non-null   int64 
 4   Bronze  1435 non-null   int64 
 5   Total   1435 non-null   int64 
 6   Year    1435 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 78.6+ KB


## 清洗数据

> 考虑之后添加部分数据来完善表格信息(反正是作为验证使用)

## 构建近五年获得奖牌总数与GDP和人口的关系

In [19]:
valid_years = [2000, 2004, 2008, 2012, 2016, 2020]
filtered_data = medal_counts[medal_counts['Year'].isin(valid_years)]

# 按国家分组聚合（包含各奖牌类型）
country_medals = filtered_data.groupby('NOC').agg({
    'Gold': 'sum',
    'Silver': 'sum',
    'Bronze': 'sum',
    'Total': 'sum'
}).reset_index()

# 添加奖牌排名（按总奖牌数降序）
country_medals = country_medals.sort_values('Total', ascending=False)
country_medals['Rank'] = range(1, len(country_medals)+1)

# 格式化输出
pd.options.display.float_format = '{:.0f}'.format
print("\n国家奖牌汇总 (2000-2020):")
print(country_medals[['Rank', 'NOC', 'Gold', 'Silver', 'Bronze', 'Total']].head(10))

# 保存结果
country_medals.to_csv('olympic_medals_2000-2020.csv', index=False)


国家奖牌汇总 (2000-2020):
     Rank            NOC  Gold  Silver  Bronze  Total
122     1  United States   242     206     196    644
21      2          China   211     136     125    472
95      3         Russia   121     105     134    360
44      4  Great Britain   117      93      95    305
42      5        Germany    80      85     104    269
4       6      Australia    80      89      95    264
39      7         France    62      80      82    224
60      8          Japan    76      61      80    217
57      9          Italy    57      59      73    189
105    10    South Korea    58      49      55    162


In [29]:
# 加载noc_codes.json，并将上面得到的country_medats中的NOC转化为相应的country代码

import json

# 加载NOC代码映射表
with open('noc_codes.json', 'r') as f:
    noc_mapping = json.load(f)

# 示例映射表结构验证
print("NOC映射表示例：", {k: noc_mapping[k] for k in list(noc_mapping)[:3]})

# 加载之前生成的奖牌数据
country_medals = pd.read_csv('olympic_medals_2000-2020.csv')

# 创建标准化的国家名称映射（处理大小写和空格）
normalized_mapping = {k.strip().lower(): v for k, v in noc_mapping.items()}

# 定义转换函数
def convert_to_noc(country_name):
    # 预处理输入名称
    clean_name = str(country_name).strip().lower()

    # 优先精确匹配
    if clean_name in normalized_mapping:
        return normalized_mapping[clean_name]

    # 常见别名处理
    alias_mapping = {
        'usa': 'USA',
        'uk': 'GBR',
        'britain': 'GBR',
        'holland': 'NED',
        'ROC':'ROC',
        'FR Yugoslavia':'FR Yugoslavia',
        'Independent Olympic Athletes':'Independent Olympic Athletes',
        'Serbia and Montenegro':'Serbia and Montenegro',
        'Macedonia':'Macedonia'
    }
    return alias_mapping.get(clean_name, pd.NA)

# 执行转换
country_medals['NOC_Code'] = country_medals['NOC'].apply(convert_to_noc)

# 检查未匹配项
unmapped = country_medals[country_medals['NOC_Code'].isna()]
if not unmapped.empty:
    print(f"发现{len(unmapped)}条未匹配记录：")
    print(unmapped['NOC'].unique())
    # 可在此处添加手动映射或记录到日志

    # 删除未匹配的数据
    for country in unmapped['NOC'].unique():
        print(country)

# 结果验证
print("\n转换结果示例：")
print(country_medals[['NOC', 'NOC_Code']].drop_duplicates().head(10))

# 可选：清理原始列
country_medals = country_medals.rename(columns={'NOC': 'CountryName'})
country_medals = country_medals[['Rank', 'NOC_Code', 'Gold', 'Silver', 'Bronze', 'Total']]

# 保存结果
country_medals.to_csv('olympic_medals_with_noc.csv', index=False)

NOC映射表示例： {'Afghanistan': 'AFG', 'Albania': 'ALB', 'Algeria': 'ALG'}
发现5条未匹配记录：
['ROC' 'FR Yugoslavia' 'Independent Olympic Athletes'
 'Serbia and Montenegro' 'Macedonia']
ROC
FR Yugoslavia
Independent Olympic Athletes
Serbia and Montenegro
Macedonia

转换结果示例：
             NOC NOC_Code
0  United States      USA
1          China      CHN
2         Russia      RUS
3  Great Britain      GBR
4        Germany      GER
5      Australia      AUS
6         France      FRA
7          Japan      JPN
8          Italy      ITA
9    South Korea      KOR
